# Lennard Jones Alpha potential test
In this example, the potential energy between two particles is compared to the analytical value.

In [1]:
import math
import sys
import subprocess
import unittest
import pandas as pd

# Run a feasst simulation with the given parameters
def run_fst(params):
    with open("launch.txt", "w") as myfile: myfile.write("""
MonteCarlo
Configuration {config_params}
Potential Model LennardJonesAlpha alpha 12
ThermoParams beta 1000000
Metropolis
Log output_file lj.csv max_precision true clear_file true
Run num_trials 1
""".format(**params))
    syscode = subprocess.call("../../../build/bin/fst < launch.txt > launch.log", shell=True, executable='/bin/bash')
    if syscode > 0: sys.exit(1)

class TestMonteCarloLJRef(unittest.TestCase):
    def test_two_particle(self):
        """Test the LJ potential against analytical calculation of two particles"""
        params = {"displacement": 1.2345}
        with open("two.xyz", "w") as myfile: myfile.write(
"""2
-1 8 8 8
0 0 0 0
1 0 0 {displacement}""".format(**params))
        run_fst({"config_params": "particle_type0 /feasst/plugin/models/particle/ljdelta.fstprt xyz_file two.xyz delta_sigma 0.5"})
        df = pd.read_csv('lj.csv')
        self.assertEqual(2, df['num_particles_of_type0'][0])

        # compute the expected analytical LJ energy
        enlj = 4*((params["displacement"]+0.5)**(-24) - (params["displacement"]+0.5)**(-12))

        # Compare the analytical results with the FEASST computed energies.
        self.assertAlmostEqual(enlj, df['LennardJonesAlpha'][0], 12)
        self.assertAlmostEqual(enlj, df['energy'][0], 12)

unittest.main(argv=[''], verbosity=2, exit=False)



test_two_particle (__main__.TestMonteCarloLJRef)
Test the LJ potential against analytical calculation of two particles ... ok

----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!